# Profiling

In [1]:
import cProfile
import pandas as pd
import re
from datetime import datetime

In [2]:
opts = [
    ("-s", ""),
    ("-r", "5"),
]
problem_dir = df_tptp["full_file_name"][1000]
problem_name = problem_dir[problem_dir.rindex("/")+1:]
args = [
    problem_dir
    # "TPTP-v9.0.0/Problems/AGT/AGT042+1.p"
    # pyres_dir + "/EXAMPLES/PUZ001+1.p"
    # "TPTP-v9.0.0/Problems/MED/MED007+1.p"
    # "TPTP-v9.0.0/Problems/CSR/CSR038+3.p"
]

with cProfile.Profile() as pr:
    pyres_fof.main(from_notebook=True, notebook_opts=opts, notebook_args=args)
    df_function_stats = pd.DataFrame(
        pr.getstats(),
        columns=["func", "ncalls", "ccalls",
                 "tottime", "cumtime", "callers"]
    )
df_function_stats["func"] = df_function_stats["func"].astype(str)
df_function_stats = \
    df_function_stats[ \
        df_function_stats["func"].str.contains(r"PyRes/(\w+)\.py") ]

func_names = []
lines = []
for value in df_function_stats["func"]:
    func_name = re.search(r'(?<=code object )(\w+)', value)
    line = re.search(r'(?<=line )(\d+)', value)
    func_names.append(func_name.group() if func_name is not None else None)
    lines.append(int(line.group()) if line is not None else None)
df_function_stats["func_name"] = func_names
df_function_stats["line"] = lines

df_function_stats = (
        df_function_stats[["func_name", "line", "ncalls", "ccalls",
                  "tottime", "cumtime", "func", "callers"]]
        .sort_values("ncalls", ascending=False)
)


datestring = (
    datetime.now().isoformat()
    .replace("2025-", "")
)
datestring = datestring[:datestring.rindex(".")]

df_function_stats.to_csv(
    str(notebook_dir)
    + f"/profiling/{datestring}_{problem_name}.csv"
    .replace(".p", "")
)
df_function_stats

NameError: name 'df_tptp' is not defined

#### Filter for alternating path functions 

In [ ]:
df_function_stats = pd.read_csv("profiling/profiling_runs.csv", index_col=0)
df_function_stats[ df_function_stats["func"].str.contains("alternatingpath.py") ]

,func_name,line,ncalls,ccalls,tottime,cumtime,func,callers
235,edge_neighb_of_subset,91,8004594,0,0.729436,0.729436,<code object edge_neighb_of_subset at 0x104c1b...,NaN
280,NaN,98,8004594,0,2.787934,2.058498,"<code object <lambda> at 0x104c79200, file ""/U...",[_lsprof.profiler_subentry(code=<code object e...
279,__init__,19,5006,0,0.000657,0.000657,"<code object __init__ at 0x104c79110, file ""/U...",NaN
278,__init__,9,1702,0,0.000144,0.000144,"<code object __init__ at 0x104c79020, file ""/U...",NaN
272,NaN,87,1702,0,0.000163,0.000163,"<code object <lambda> at 0x104c52b30, file ""/U...",NaN
284,get_neighbours,95,1599,0,4.077462,1.289528,"<code object get_neighbours at 0x104c983b0, fi...",[_lsprof.profiler_subentry(code=<code object <...
270,get_all_nodes,73,1,0,0.000036,0.000036,"<code object get_all_nodes at 0x104c52230, fil...",NaN
163,construct_betweenclause_edges,57,1,0,5.855480,0.215930,<code object construct_betweenclause_edges at ...,[_lsprof.profiler_subentry(code=<code object t...
196,nodes_to_clauses,76,1,0,0.000007,0.000006,"<code object nodes_to_clauses at 0x1043e8030, ...",[_lsprof.profiler_subentry(code=<code object _...
199,get_rel_neighbourhood,109,1,0,4.080163,0.002488,<code object get_rel_neighbourhood at 0x1043f3...,[_lsprof.profiler_subentry(code=<code object c...
